In [26]:
import sys
import os
import importlib
import numpy as np
import time
import pickle
import json
import neat
import imageio

# Add the parent directory to the Python path
sys.path.append(os.path.abspath('../'))
import environment 
import utils
importlib.reload(utils), importlib.reload(environment)

(<module 'utils' from '/Users/lorenzoleuzzi/Documents/GitHub/lifelong_evolutionary_swarms/utils.py'>,
 <module 'environment' from '/Users/lorenzoleuzzi/Documents/GitHub/lifelong_evolutionary_swarms/environment.py'>)

In [122]:
SAVE = True
NAME = "population"
FPS = 30
POP = True
COLOR_MAP = {
    3: "red",
    4: "blue",
    5: "green",
    6: "yellow",
    7: "purple",
    8: "white",
    9: "#0072BD", # cyan
    10: "black"
}

In [138]:
# Load a controller
path = f"/Users/lorenzoleuzzi/Library/CloudStorage/OneDrive-UniversityofPisa/lifelong_evolutionary_swarms/results_gecco4/snz_baselines/neat_500_200_300_5_20_10/seed17/static3_drift55"
with open(f"{path}/best_genome.pkl", "rb") as f:
    controller = pickle.load(f)
with open(f"{path}/neat_config.pkl", "rb") as f:
    config_neat = pickle.load(f)
with open(f"{path}/info.json", "r") as f:
    info_exp = json.load(f)
with open(f"{path}/population.pkl", "rb") as f:
    population = pickle.load(f)
# with open(f"{path}/env.pkl", "rb") as f:
#     env = pickle.load(f) # does not work because different versions of Python between local and remote
env = environment.SwarmForagingEnv(n_agents = info_exp["n_agents"], n_blocks = info_exp["n_blocks"], colors=[3, 4, 5, 6],
                                   season_colors=info_exp["season_colors"],
                           target_color=info_exp["target_color"], duration=info_exp["episode_duration"])
if POP:
    id_ret_pop = 0
    prev_target = info_exp["prev_target_colors"][-1]
    controller = population.population[info_exp[f"id_retention_pop_{prev_target}"]]

net = neat.nn.FeedForwardNetwork.create(controller, config_neat)

In [127]:
env.change_season([5,6], 5)

In [139]:
env.change_season([3,4], 3)

In [141]:
fitnesses = []
corrects = []
wrongs = []

episodes_info = {}
for i in range(3):
    
    frames = []
    obs, _ = env.reset(seed=None)
    fitness = 0
    frames.append(env.render(True, False))
    while True:
        
        nn_inputs = env.process_observation(obs)
        
        nn_outputs = np.array([net.activate(nn_input) for nn_input in nn_inputs])
        actions = (2 * nn_outputs - 1) * env.max_wheel_velocity # Scale output sigmoid in range of wheel velocity

        obs, reward, done, truncated, info = env.step(actions)
        
        frames.append(env.render(True, False))
        
        fitness += reward
        if done or truncated:
            print(f"Done: {done}, Truncated: {truncated}, Fitness: {fitness}, correct: {len(info['correct_retrieves'])}, wrong: {len(info['wrong_retrieves'])}")
            episodes_info[i] = ((fitness, len(info['correct_retrieves']), len(info['wrong_retrieves'])))
            break
    
    if SAVE == True:
        # Create folder if it does not exist
        if not os.path.exists(f"episodes/{NAME}"):
            os.makedirs(f"episodes/{NAME}")
        imageio.mimsave(f"episodes/{NAME}/target_{COLOR_MAP[env.target_color]}_{i}.gif", frames, fps = FPS)
    
    fitnesses.append(fitness)
    corrects.append(len(info['correct_retrieves']))
    wrongs.append(len(info['wrong_retrieves']))

episodes_info["mean"] = (np.mean(fitnesses), np.mean(corrects), np.mean(wrongs))
if SAVE == True:
    with open(f"episodes/{NAME}/episodes_info_{COLOR_MAP[env.target_color]}.json", "w") as f:
        json.dump(episodes_info, f, indent=4)
print(np.mean(fitnesses), np.mean(corrects), np.mean(wrongs))

Done: False, Truncated: True, Fitness: 13.0, correct: 5, wrong: 2
Done: False, Truncated: True, Fitness: 19.0, correct: 7, wrong: 2
Done: False, Truncated: True, Fitness: 5.0, correct: 2, wrong: 1
12.333333333333334 4.666666666666667 1.6666666666666667
